In [72]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pulp as plp
import networkx as nx
from ALB_instance_tools import *
from collections import namedtuple


In [73]:
def make_scenario_tree(n_takts, entry_probabilities):
    """
    Creates a scenario tree for the given number of takts, instance and entry probabilities.
    """
    # Create a directed graph
    G = nx.DiGraph()
    # Add the root node
    G.add_node('R', stage=0, scenario=0)
    #Create a list of final sequences
    final_sequences = {}
    def add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability=1, parent=0, sequence = [], current_stage=0, counter=[0]):
        if current_stage == n_takts:
            final_sequences[counter[0]] = {'sequence':sequence, 'probability': probability}
            counter[0] += 1
            return
        else:
            for model, prob in entry_probabilities.items():
                new_sequence = sequence.copy()
                new_sequence.append(model)
                node_name = str(parent)+ str(model) 
                graph.add_node(node_name, stage = current_stage, scenario = new_sequence)
                graph.add_edge(parent, node_name, probability = probability)
                add_nodes(n_takts, entry_probabilities, graph, final_sequences, probability* prob,node_name, new_sequence, current_stage+1)
    add_nodes(n_takts, entry_probabilities, G, final_sequences, parent='R')
    return G, final_sequences
# NO_takts = 4
# scenario_tree_graph, final_sequences = make_scenario_tree(NO_takts, {'A':0.75, 'B':0.25})
# final_sequences

In [74]:
def check_scenarios(prod_sequence1,prod_sequence2,t):
    '''compares two production sequences up to time t and returns true if they are the same'''
    if prod_sequence1[:t+1] == prod_sequence2[:t+1]:
        return True
    else:
        return False


In [75]:
def add_non_aticipation(prob, w, w_prime , prod_sequences, model_instance, x_wsoj, sequence_length, num_stations):
    '''adds the non participation constraint for two scenarios w and w_prime'''
    #We go backwards in time, look for first time t where the sequences are the same
    for t in reversed(range(sequence_length)):
        if check_scenarios(prod_sequences[w]['sequence'], prod_sequences[w_prime]['sequence'], t):
            for j in range(t+1):
                model = prod_sequences[w]['sequence'][j]
                max_station = min(t-j+1, num_stations)    
                for s in range(max_station):
                    for o in range(model_instance[model]['num_tasks']):
                        prob += (x_wsoj[w][s][o][j] == x_wsoj[w_prime][s][o][j], 
                                f'anti_ww*soj_{w}_{w_prime}_{s}_{o}_{j}')
            return


In [76]:


def stochastic_problem_linear_labour(model_instance, equipment_instance,no_tasks, NO_WORKERS, NO_STATIONS,takt_time, sequence_length, prod_sequences, worker_cost =100):
    print('Writing problem')
    print('Number of tasks:', no_tasks, 'Number of workers:', NO_WORKERS, '\n','Number of stations:', NO_STATIONS, 'Takt time:', takt_time, 'Sequence length:', sequence_length, 'worker_cost:', worker_cost)
    workers = list(range(0, NO_WORKERS+1))
    stations = list(range(NO_STATIONS))
    c_se = equipment_instance['equipment_prices']
    R_oe = equipment_instance['equipment_matrix']
    equipment = list( range(R_oe.shape[1]))
    takts = list(range(sequence_length+NO_STATIONS-1))
    u_se = plp.LpVariable.dicts('u_se', (stations, equipment), lowBound=0, cat='Binary')
    b_wtsl = plp.LpVariable.dicts('b_wtsl', (prod_sequences.keys(), takts, stations, workers), lowBound=0, cat='Binary') 
    #TODO: maybe make this dictionary work different number of tasks for each model
    x_wsoj = plp.LpVariable.dicts('x_wsoj', (prod_sequences.keys(), stations, range(no_tasks), range(sequence_length) ), lowBound=0, cat='Binary')
    #Defining LP problem
    prob = plp.LpProblem("Stochastic_problem", plp.LpMinimize)
    #Objective function
    prob += (plp.lpSum([c_se[s][e]*u_se[s][e]
                         for s in stations
                           for e in equipment]
                      +
                      [prod_sequences[w]['probability']/sequence_length * worker_cost* l * b_wtsl[w][t][s][l]
                         for w in prod_sequences.keys()
                           for t in range(0,sequence_length) 
                           for s in stations for l in workers]),
                "Total cost")
    #Constraints
    #constraints work over all scenarios
    # for w in final_sequences.keys():
        #Constraint 1 -- can only assign l number of workers to a station for a given scenario and stage
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                prob += (plp.lpSum([b_wtsl[w][t][s][l] for l in workers]) == 1, f'b_wtsl_{w}_{t}_{s}')
        #Constraint 2 all tasks must be assigned to a station
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            #Not strictly necessary if all models have the same number of tasks, could have just looped over no tasks
            #but this is more general
            for o in range(model_instance[model]['num_tasks']): 
                prob += (plp.lpSum([x_wsoj[w][s][o][j] for s in stations]) == 1, f'x_wsoj_{w}_s_{o}_{j}')
        #Constraint 3 -- sum of task times for assigned tasks must be less than takt time times the number of workers for a given station
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                #Get the model at the current scenario, stage, and station
                if 0<= t-s < sequence_length:
                    j = t-s
                    model = prod_sequences[w]['sequence'][j]
                    task_times = model_instance[model]['task_times']
                    prob += (plp.lpSum([task_times[o]*x_wsoj[w][s][int(o)-1][j] 
                                        for o in task_times]) 
                                        <= 
                                        takt_time*plp.lpSum([l * b_wtsl[w][t][s][l] for l in workers]), f'task_time_wts_{w}_{t}_{s}')

    #Constraint 4 -- tasks can only be assigned to a station with the correct equipment
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for s in stations:
                for o in range(model_instance[model]['num_tasks']):
                    prob += x_wsoj[w][s][o][j] <= plp.lpSum([R_oe[o][e]*u_se[s][e] for  e in equipment]), f'equipment_wsoj_{w}_{s}_{o}_{j}'
        #Constraint 5 -- precedence constraints
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for (pred, suc) in model_instance[model]['precedence_relations']:
                prob += (plp.lpSum([ (s+1)  * x_wsoj[w][s][int(pred)-1][j] for s in stations])
                         <=  
                         plp.lpSum([ (s+1)  * x_wsoj[w][s][int(suc)-1][j] for s in stations]), 
                         f'task{pred} before task{suc} for model{model}, item {j} seq {w}' )
        #Constraint 6 -- non-anticipativity constraints
    for w in prod_sequences.keys():
        for w_prime in prod_sequences.keys():
            if w_prime > w:
                add_non_aticipation(prob, w, w_prime , prod_sequences, model_instance, x_wsoj, sequence_length, NO_STATIONS)
                # for j in reversed(range(sequence_length)):
                #     if check_scenarios(prod_sequences[w]['sequence'], prod_sequences[w_prime]['sequence'], j):
                #         model = prod_sequences[w]['sequence'][j]
                #         print('w', w, 'w_prime', w_prime)
                #         print('j', j, 'max_station', NO_STATIONS-j, 'model', model)
                #         for s in range(NO_STATIONS - j ):
                #             for o in range(model_instance[model]['num_tasks']):
                #                 prob += (x_wsoj[w][s][o][j] == x_wsoj[w_prime][s][o][j], 
                #                         f'anti_ww*soj_{w}_{w_prime}_{s}_{o}_{j}')

                
    return prob
#prob = stochastic_problem_linear_labour(test_instances, equipment_instance[1],no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, NO_takts, final_sequences, worker_cost=WORKER_COST)                                


In [77]:
#TODO - add secondary objective that tasks should be at the same station for the same model
def generate_task_assignments_df_stochastic(pulp_prob, sequences, instances):
    '''Shows task assignments for fixed and model dependent task assignment'''
    task_assignments = []
    labor_assignments = []
    for v in pulp_prob.variables():
        print(v.name, v.varValue)
        if round(v.varValue) > 0:
            
            if 'x_wsoj' in v.name:
                sequence = int(v.name.split('_')[2])
                item = int(v.name.split('_')[5])
                model = sequences[sequence]['sequence'][item]
                #change the task number to match with the instances
                task = str(int(v.name.split('_')[4])+1)
                task_time = instances[model]['task_times'][task]
                assignment = {'scenario':v.name.split('_')[2], 'station': v.name.split('_')[3],'sequence_loc':item, 'model':model  , 'task': task, 'task_times': task_time}
                task_assignments.append(assignment)
            elif 'b_wtsl' in v.name:
                model = sequences[int(v.name.split('_')[2])]['sequence'][int(v.name.split('_')[4])]
                labor = {'scenario':v.name.split('_')[2], 'stage':v.name.split('_')[3], 'station': v.name.split('_')[4], 'model':model, 'workers': int(v.name.split('_')[5]) }
                labor_assignments.append(labor)

    #turns task_assignments into a dataframe
    task_assignments_df = pd.DataFrame(task_assignments)
    labor_assignments_df = pd.DataFrame(labor_assignments)
    #concatenates the 'task' column in task_assignments_df if the 'station' and 'model' columns are the same
    task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()
    labor_assignments_df['sequence_loc'] = labor_assignments_df['stage'].astype(int) - labor_assignments_df['station'].astype(int)
    labor_assignments_df = labor_assignments_df[labor_assignments_df['sequence_loc'] >= 0]
    return task_assignments_df, labor_assignments_df
#task_assignment, labor_assignment = generate_task_assignments_df_stochastic(prob, prod_sequences, test_instances)
#task_assignment

In [78]:
# instance_dicts = [
#    {'fp': "SALBP_benchmark/debugging_ds/instance_n=5_1.alb",'name':'A', 'probability':0.75},
#    {'fp': "SALBP_benchmark/debugging_ds/instance_n=5_2.alb",'name':'B', 'probability':0.25}
# ]

# instance_dicts = [
#    {'fp': "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",'name':'A', 'probability':0.75},
#    {'fp': "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",'name':'B', 'probability':0.25}
# ]

instance_list = [ "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",
   "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",
   "SALBP_benchmark/small data set_n=20/instance_n=20_3.alb",]
#test_instances = create_instance_pair_stochastic(instance_dicts)


NO_EQUIPMENT = 4
seed = 1
NO_WORKERS =4
NO_STATIONS = 3
WORKER_COST = 500
TAKT_TIME = 300
NO_TAKTS = 4
MODEL_MIXTURES = {'A':0.75, 'B':0.25}
#TODO Make this work for more than 2 models
def pair_instances(instance_list, MODEL_MIXTURES):
      instance_groups = []
      for i in range(len(instance_list)):
         for j in range(i+1, len(instance_list)):
            a_name = list(MODEL_MIXTURES.keys())[0]
            b_name = list(MODEL_MIXTURES.keys())[1]
            instance_groups.append([{'fp':instance_list[i], 'name':a_name, 'probability':MODEL_MIXTURES[a_name]},
                                   {'fp':instance_list[j], 'name':b_name, 'probability':MODEL_MIXTURES[b_name]}])
      return instance_groups

def multi_run(instance_list, NO_EQUIPMENT,  NO_WORKERS, NO_STATIONS, WORKER_COST, TAKT_TIME, NO_TAKTS, MODEL_MIXTURES, seed):
   instance_groups = pair_instances(instance_list, MODEL_MIXTURES)
   print(instance_groups)
   test_results = []
   group_counter = 0
   test_instances = []
   for group in instance_groups:
      test_instance = create_instance_pair_stochastic(group)
      test_instances.append(test_instance)
      #create equipment
      print('creating equipment')
      all_tasks = get_task_union(test_instance, 'A', 'B')
      no_tasks = len(all_tasks)
      c_se, r_oe = generate_equipment(NO_EQUIPMENT, NO_STATIONS, all_tasks, seed = seed)
      equipment_instance = {1: {'equipment_prices': c_se, 'equipment_matrix': r_oe}}
      #create scenario tree
      print('generating scenario tree')
      scenario_tree_graph, final_sequences = make_scenario_tree(NO_TAKTS, MODEL_MIXTURES)
      print('defining problem')
      prob = stochastic_problem_linear_labour(test_instance, equipment_instance[1],no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, NO_TAKTS, final_sequences, worker_cost=WORKER_COST)
      print('solving problem')
      solver = plp.GUROBI_CMD()
      prob.solve(solver=solver)
      print('writing results')
      task_assignment, labor_assignment = generate_task_assignments_df_stochastic(prob, final_sequences, test_instance)
      task_seq = task_assignment[['scenario','station', 'task','task_times', 'sequence_loc']]
      #merging labor and task sequence dataframes
      labor_task_seq = pd.merge(labor_assignment, task_seq, on=['scenario','station', 'sequence_loc'], how='left')
      task_assignment.to_csv(f'task_assignment_group_{group_counter}.csv', sep=' ')
      labor_task_seq.to_csv(f'labor_assignment_group_{group_counter}.csv', sep=' ')
      group_counter += 1
multi_run(instance_list, NO_EQUIPMENT,  NO_WORKERS, NO_STATIONS, WORKER_COST, TAKT_TIME, NO_TAKTS, MODEL_MIXTURES, seed)

[[{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_1.alb', 'name': 'A', 'probability': 0.75}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_2.alb', 'name': 'B', 'probability': 0.25}], [{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_1.alb', 'name': 'A', 'probability': 0.75}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_3.alb', 'name': 'B', 'probability': 0.25}], [{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_2.alb', 'name': 'A', 'probability': 0.75}, {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_3.alb', 'name': 'B', 'probability': 0.25}]]
creating equipment
generating scenario tree
defining problem
Writing problem
Number of tasks: 20 Number of workers: 4 
 Number of stations: 3 Takt time: 300 Sequence length: 4 worker_cost: 500
solving problem
Set parameter LogFile to value "gurobi.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright 

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_40498/1208923960.py:28: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


solving problem
Set parameter LogFile to value "gurobi.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/d55caf659da64e85a9ea8c37e0569691-pulp.lp
Reading time = 0.02 seconds
Total_cost: 9248 rows, 5292 columns, 39200 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9248 rows, 5292 columns and 39200 nonzeros
Model fingerprint: 0x9b5027a4
Variable types: 0 continuous, 5292 integer (5292 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e-01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6216 rows and 3228 columns
Presolve time: 0.06s
Presolved: 3032 rows, 2064 columns, 13974 nonzeros
Variable types: 0 continuous, 2064 integer 

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_40498/1208923960.py:28: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


solving problem
Set parameter LogFile to value "gurobi.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/13e9009f45524b14813502fcac5aa418-pulp.lp
Reading time = 0.02 seconds
Total_cost: 9344 rows, 5292 columns, 39776 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9344 rows, 5292 columns and 39776 nonzeros
Model fingerprint: 0x37b5bedc
Variable types: 0 continuous, 5292 integer (5292 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [5e-01, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6408 rows and 3325 columns
Presolve time: 0.07s
Presolved: 2936 rows, 1967 columns, 12854 nonzeros
Variable types: 0 continuous, 1967 integer 

/var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/ipykernel_40498/1208923960.py:28: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['scenario','station', 'sequence_loc','model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


In [79]:
MODEL_MIXTURES.keys()

dict_keys(['A', 'B'])